In [ ]:
#imports
import time
import tweepy
import numpy as np
import pandas as pd
import seaborn as sns


# Inputs: 
## Below is the only cell you need to make changes to. 

In [ ]:
#input variables

#For whome? if self, give your screenname below
ScreenName='balajis'

#First time load?
FirstLoadFlag=True

############### Twitter API details #################################################
#Keys
ConsumerKey= "XX"
ConsumerSecret= "XX"
AccessKey= "XX"
AccessSecret="XX"


# Authenticate to Twitter
auth = tweepy.OAuthHandler(ConsumerKey, ConsumerSecret)
auth.set_access_token(AccessKey, AccessSecret)

# Create API object
api = tweepy.API(auth)

############################## end #################################################


############### Followers Update parameters #######################################
#maximum increase in users per day. Used to check for new followers that get added after initial load
MaxFollowerUpPerDay=500

############################## end #################################################


################ Filter parameters for choosing followers ##########################
#Keywords in Bio (keep them empty if you want to keep all followers for DM irrespective of Keywords)
#only 3 keywords given to have less latency
BioKW1=''
BioKW2=''
BioKW3=''


#minimum follower count that your followers need to have
MinFollowerCountPerFollower=1000

############################## end #################################################


####################Direct message inputs #########################################
#DM text contents
DMtext='Hi, how are you. Send me your emailid.'

ScreenNameExcludes=['databaaj', 'sandeepchandola']

############################## end #################################################

## Do not do make changes to any cells below this line. Just execute them.

In [ ]:
#backup data

if not FirstLoadFlag:
    ids=pd.read_csv('ids.csv')
    ids.to_csv('backup_ids.csv')

    meta=pd.read_csv('meta.csv')
    meta.to_csv('backup_meta.csv')

In [ ]:
# Status Visual


if not FirstLoadFlag:
    user = api.get_user(ScreenName)
    FollowersCount=user.followers_count

    ids=pd.read_csv('ids.csv')
    FollowersCountDB=len(ids)

    meta=pd.read_csv('meta.csv')
    FollowersCountWithMeta=len(meta)

    DMsentSuccess=len(meta[meta.DMsent=='Success'])
    DMsentFail=len(meta[meta.DMsent=='Fail'])
    DMnotsent=len(meta[meta.DMsent=='False'])

    chart_list=[('FollowersCount', FollowersCount, ''), ('FollowersCountDB', FollowersCountDB, ''), ('FollowersCountWithMeta', FollowersCountWithMeta, ''), ('DMsent', DMsentSuccess, DMsentFail), ('DMnotsent',DMnotsent,'')]
    
    chart_df=pd.DataFrame(chart_list, columns=['Metrics', 'Success', 'Fail'])
    chart_df.sort_index(inplace=True, ascending=False)
    chart_df.set_index('Metrics', inplace=True)
    print (chart_df)


    chart_df.plot(kind='barh', stacked=True, figsize=[10,7])

In [ ]:
#Update Follower IDS list

#list to hold new ids
ids = []

def loop_to_get_ids(users):
    ids=[]
    print (time.time())
    while True:
        try:
            user = next(users)
            ids+=[user]
        except tweepy.TweepError:
            print ('time limit, waiting for 15 mins ------------------------')
            print ('limit len:', len(ids))
            time.sleep(60*15)
            user = next(users)
            ids+=[user]
        except StopIteration:
            break

    print (time.time())
    return ids

if FirstLoadFlag:
    users = tweepy.Cursor(api.followers_ids, screen_name=ScreenName).items()

    ids=loop_to_get_ids(users)

    ids_df=pd.DataFrame(ids, columns=['ids'])
    ids_df['PulledTime']=time.time()
    # ids_df.sort_values('ids', inplace=True)


    ids_df.to_csv('ids.csv', encoding='utf-8')

else:
    users = tweepy.Cursor(api.followers_ids, screen_name=ScreenName).items(MaxFollowerUpPerDay)
    
    ids_df=pd.read_csv('ids.csv')
    ids_set=set(ids_df['ids'].tolist())
#     lastfollower=ids_df.loc[0,'ids']
    
    
    new_ids=loop_to_get_ids(users)
    new_ids_set=set(new_ids)
    
    unique_new_ids=list(new_ids_set-ids_set)
    
    new_ids_df=pd.DataFrame(unique_new_ids, columns=['ids'])
    new_ids_df['pulledtime']=time.time()
    
    new_ids_df.to_csv('ids.csv', mode='a', encoding='utf-8')

In [ ]:
#update metadata (screenName, followersCount, bio, etc.)

ids_df=pd.read_csv('ids.csv')

ids=ids_df['ids'].tolist()


if FirstLoadFlag:
    id_counts=[]
    for lim in np.arange(0,len(ids),100):
        if lim % 1000 == 0:
            print (lim," records done.")
        user_objs=api.lookup_users(user_ids=ids[lim:lim+100])
        id_counts+=[(obj.id, obj.screen_name, obj.followers_count, obj.description, 'No') for obj in user_objs]

    meta_df=pd.DataFrame(id_counts, columns=['id', 'screenname', 'count', 'description', 'DMsent'])
    meta_df.to_csv('meta.csv', encoding = 'utf-8')

else:
    ids=ids_df.loc[ids_df['pulledtime']==max(ids_df['pulledtime']), 'ids'].tolist()
    id_counts=[]
    for lim in np.arange(0,len(ids),100):
        if lim % 1000 == 0:
            print (lim," records done.")
        user_objs=api.lookup_users(user_ids=ids[lim:lim+100])
        id_counts+=[(obj.id, obj.screen_name, obj.followers_count, obj.description, 'No') for obj in user_objs]

    new_meta_df=pd.DataFrame(id_counts, columns=['id', 'screenname', 'count', 'description', 'DMsent'])
    new_meta_df.to_csv('meta.csv', mode='a', encoding = 'utf-8')



In [ ]:
#Send DMs for today

#Filter and sort before DM
meta_df=pd.read_csv('meta.csv')

# print (meta_df.head())

#filter
#filter followers with keywords in bio
if not(BioKW1=='' and BioKW2=='' and BioKW3==''):
    meta_df=meta_df[(meta_df.description.str.contains(BioKW1)) | (meta_df.description.str.contains(BioKW2)) | (meta_df.description.str.contains(BioKW3))]

#filter followers with followers more than certian number
meta_df=meta_df[meta_df['count']>MinFollowerCountPerFollower]

#filtering to following that were not sent DMs earlier
meta_df=meta_df[meta_df['DMsent']=='No']

#filtering our followers that are marked for exlusion
meta_df=meta_df[~meta_df['screenname'].isin(ScreenNameExcludes)]

#sort
meta_df.sort_values('count', ascending= False, inplace=True)

#send DM to top 1000
print ('Screen Name', ': ', 'Followers Count')
failed=[]
for i,each in meta_df[:1000].iterrows():
    print (each['screenname'], ': ', each['count'])
    try:
        api.send_direct_message(each['id'], DMtext)
    except Exception as e:
        excep = str(e)
        failed+=[(each['screenname'], excep)]

#write DM sent report
failed_df=pd.DataFrame(failed, columns=['screenname', 'failedreason'])
failed_df.to_csv('DMsentReport.csv', encoding='utf-8')